In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import random
from sklearn.cross_validation import train_test_split
from datetime import datetime
tf.logging.set_verbosity(tf.logging.INFO)

import tensorlayer as tl

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
width = 640
height = 480
channel = 3
total_pix = width * height * channel

sess = tf.InteractiveSession()
x = tf.placeholder("float", [None, total_pix])
y = tf.placeholder(np.int64, [None, ])

In [ ]:
input_x = tf.reshape(x, [-1, 480, 640, 3])

input_layer = tl.layers.InputLayer(input_x, name='input')

conv1 = tl.layers.Conv2d(input_layer,
                         48,
                         [5, 5],
                         [1, 1],
                         act=tf.nn.relu,
                         padding='SAME',
                         name='conv1')
pool1 = tl.layers.MaxPool2d(conv1, [2, 2], [2, 2], padding='SAME', name='pool1')


conv2 = tl.layers.Conv2d(pool1,
                         96,
                         [5, 5],
                         [1, 1],
                         act=tf.nn.relu,
                         padding='SAME',
                         name='conv2')
pool2 = tl.layers.MaxPool2d(conv2, [2, 2], [2, 2], padding='SAME', name='pool2')


conv3 = tl.layers.Conv2d(pool2,
                         96,
                         [5, 5],
                         [1, 1],
                         act=tf.nn.relu,
                         padding='SAME',
                         name='conv3')
pool3 = tl.layers.MaxPool2d(conv3, [2, 2], [2, 2], padding='SAME', name='pool3')


conv4 = tl.layers.Conv2d(pool3,
                         96,
                         [5, 5],
                         [1, 1],
                         act=tf.nn.relu,
                         padding='SAME',
                         name='conv4')
pool4 = tl.layers.MaxPool2d(conv4, [2, 2], [2, 2], padding='SAME', name='pool4')


conv5 = tl.layers.Conv2d(pool4,
                         96,
                         [5, 5],
                         [1, 1],
                         act=tf.nn.relu,
                         padding='SAME',
                         name='conv5')
pool5 = tl.layers.MaxPool2d(conv5, [2, 2], [2, 2], padding='SAME', name='pool5')


pool5_flat = tl.layers.FlattenLayer(pool5, name='flatten')
dense1 = tl.layers.DenseLayer(pool5_flat, 1024, act=tf.nn.relu, name='dense1')
drop1 = tl.layers.DropoutLayer(dense1, keep=0.5, name='drop1')
output_layer = tl.layers.DenseLayer(drop1, 2, act=tf.nn.relu, name='output')

In [ ]:
logits = output_layer.outputs

y_prob = tf.nn.softmax(logits, name="probability")
y_op = tf.argmax(input = logits, axis=1)
#loss = tf.losses.sparse_softmax_cross_entropy(logits=logits, labels=y)
loss = tl.cost.cross_entropy(output=logits, target=y, name='loss')

correct_prediction = tf.equal(y_op, y)
acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#acc2 = tf.metrics.accuracy(labels=y, predictions=y_op)

In [ ]:
train_params = output_layer.all_params
train_op = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss=loss,
                                                                           global_step=tf.train.get_global_step(),
                                                                          var_list=train_params)


# Data loading

In [ ]:
def data_process(d_dir, split_rate):
    
    num_class = len(d_dir.keys())

    out = {}
    

    for i in range(num_class):
        x_train, x_test = train_test_split(d_dir[i], test_size=split_rate)
        
        if i == 0:
            train_temp = x_train
            test_temp = x_test
        else:
            train_temp.extend(x_train)
            test_temp.extend(x_test)
    
    random.shuffle(train_temp)
    out['train'] = train_temp
    random.shuffle(test_temp)
    out['test'] = test_temp
    
    return out
        

In [ ]:
image_dir1 = '/dataset/'
class_name1 = ['untorn', 'torn']

data_dir1 = {}
for i in range(len(class_name1)):
    temp_dir = image_dir1 + class_name1[i]
    img_files = [[os.path.join(temp_dir, f), np.int16(i)] for f in os.listdir(temp_dir) if f.endswith('.png')]
    data_dir1[i] = img_files
print(len(data_dir1[0]))

In [ ]:
tr_data_dir = data_process(data_dir1, 0.2)

In [ ]:
tr_data = np.zeros((len(tr_data_dir['train']), total_pix), dtype=np.float32)
tr_label = np.zeros((len(tr_data_dir['train']), 1), dtype=np.int32)

for i in range(len(tr_data_dir['train'])):
    img = cv2.imread(tr_data_dir['train'][i][0])
    tr_data[i][:] = img.flatten()
    tr_label[i] = tr_data_dir['train'][i][1]

In [ ]:
ev_data = np.zeros((len(tr_data_dir['test']), total_pix), dtype=np.float32)
ev_label = np.zeros((len(tr_data_dir['test']), 1), dtype=np.int32)

for i in range(len(tr_data_dir['test'])):
    img = cv2.imread(tr_data_dir['test'][i][0])
    ev_data[i][:] = img.flatten()
    ev_label[i] = tr_data_dir['test'][i][1]

In [ ]:
today = datetime.today().strftime("%Y-%m-%d-%H-%M-%S")
print(today)

# Training

In [ ]:
batch_s = 16
epochs = 45
tr_steps = int(tr_data.shape[0]/batch_s)
print(tr_steps)

In [ ]:
tl.layers.initialize_global_variables(sess)
output_layer.print_params()
output_layer.print_layers()

In [ ]:
# For cross entropy with logits
count = 1
for i in range(epochs):
    for j in range(tr_steps):
        
        if j == (tr_steps-1):
            temp = tr_label[j*batch_s:]
            temp2 = temp.reshape(len(temp),)
            feed_dict = {x: tr_data[j*batch_s:], y: temp2}
        
        else:
            temp = tr_label[j*batch_s:(j+1)*batch_s]
            temp2 = temp.reshape(batch_s,)
            feed_dict = {x: tr_data[j*batch_s:(j+1)*batch_s], y: temp2}
            
        
        
        feed_dict.update(output_layer.all_drop)
        sess.run(train_op, feed_dict=feed_dict)
        
        if count % 10 == 0:
            dp_dict = tl.utils.dict_to_one(output_layer.all_drop)
            feed_dict.update(dp_dict)
            error, accuracy = sess.run([loss, acc], feed_dict=feed_dict)
            print("Steps: %d, loss: %f, accuracy: %f" % (count, error, accuracy))
        
        count = count + 1

In [ ]:
count = 1
for i in range(epochs):
    for j in range(tr_steps):
        
        if j == (tr_steps-1):
            feed_dict = {x: tr_data[j*batch_s:], y: tr_label[j*batch_s:]}
        
        else:
            feed_dict = {x: tr_data[j*batch_s:(j+1)*batch_s], y: tr_label[j*batch_s:(j+1)*batch_s]}
            
        
        
        feed_dict.update(output_layer.all_drop)
        sess.run(train_op, feed_dict=feed_dict)
        
        if count % 10 == 0:
            dp_dict = tl.utils.dict_to_one(output_layer.all_drop)
            feed_dict.update(dp_dict)
            error, accuracy = sess.run([loss, acc], feed_dict=feed_dict)
            print("Steps: %d, loss: %f, accuracy: %f" % (count, error, accuracy))
        
        count = count + 1

In [ ]:
err = np.zeros((len(ev_label), 1))
ac = np.zeros((len(ev_label), 1))

for k in range(len(ev_label)):
    dp_dict = tl.utils.dict_to_one(output_layer.all_drop)
    feed_dict = {x: ev_data[k:k+1], y: ev_label[k:k+1]}
    feed_dict.update(dp_dict)
    t1, t2 = sess.run([loss, acc2], feed_dict=feed_dict)
    err[k] = t1
    ac[k] = t2

In [ ]:
print(np.mean(ac))